# `Project Outline Berlin`


## Objective:

> Make a `Snakemake pipeline` that outputs a dataframe or an Excel sheet that show which amino acid is present at a given position per lineage and segment of influenza viruses. Hereby trying to make a somewhat more flexible tool, so it could also be applied to other lineages than used here such as H5N1 or H7N9 lineages.
>
> FASTA files may contain nucloetide seqeunces from segments of influenza virus subtypes `H1N1pdm`, `H3N2`, `B-Vic` and `B-Yam` downloaded from GISAID.
>
> This FASTA file will first be processed by `Nextclade` (for `HA` & `NA` segments) and `Nextalign` (for `PB2`, `PB1`, `PA`, `NP`, `MA` & `NS` segments). Here, input nucleotide sequences are aligned to their appropriate reference genomes. Translated alignments are truncated to the sequence length representing the viral functional protein.
> 
> `Nextclade` and `Nextalign` both output aligned nucleotide sequences and aligned amino acid sequences that are truncated according to a correct numbering used for annotation.
>
>   **Reference for HA numbering:**<br/>
>Burke DF and Smith DJ (2014) <br/>A standardised numbering for all subtypes of Influenza A hemaggluttin (HA) sequences. <br/>Plos One 9(11): e112302 <br/>DOI: 10.1371/journal.pone.0112302 <br/>[Reference table](https://antigenic-cartography.org/surveillance/evergreen/HAnumbering/)
>
> Python scripts will than process the output files from `Nextclade` and `Nextalign` and finally output tables where rows are represented by the input sequence names and where columns represent correctly numbered amino acids posititions within the analysed segment.

## Input and output files

#### INPUT FILES

1. Downloaded FASTA files from GISAID with nucleotide sequences and headers that are formatted like:

```
Isolate ID | Isolate name | Type | Lineage | Segment | Passage details/history | Collection date
```



**Example fasta header:**    
```
>EPI_ISL_342146|A/Netherlands/00322/2019|A_/_H1N1|pdm09|HA|Original|2019-02-06
```



2. Lists of amino acid positions per viral protein sequences grouped by subtype and gene segment.       
    Applying an Excel spreadsheet as a provided input might give more flexibility in use of the final pipeline.


    *Example input table:*

lineage | segment | position
--------|---------|---------
H1N1pdm | HA      | 156
H1N1pdm | HA      | 158
H1N1pdm | HA      | 162
H1N1pdm | NA      | 274
H1N1pdm | PA      | 31
H3N2    | HA      | 145
H3N2    | HA      | 184
H3N2    | HA      | 185
etc...  | etc...  | etc...




3. Datasets that are used for `Nextclade` and `Nextalign`to run:

- **Nextalign input files:**
    - `_nextalign/data/flu_h1n1pdm_pa/genemap.gff`
    - `_nextalign/data/flu_h1n1pdm_pa/reference.fasta`
    - `_nextalign/data/flu_h1n1pdm_pa/sequences.fasta`

<br/>

- **Nextclade input files:**
    - `_nextclade/data/flu_h1n1pdm_ha/primers.cvs`
    - `_nextclade/data/flu_h1n1pdm_ha/genemap.gff`
    - `_nextclade/data/flu_h1n1pdm_ha/qc.json`
    - `_nextclade/data/flu_h1n1pdm_ha/reference.fasta`
    - `_nextclade/data/flu_h1n1pdm_ha/sequences.fasta`
    - `_nextclade/data/flu_h1n1pdm_ha/tag.json`
    - `_nextclade/data/flu_h1n1pdm_ha/tree.json`
    - `_nextclade/data/flu_h1n1pdm_ha/virus_properties.json`



#### OUTPUT FILES

- `Nextclade` and `Nextalign` should give outputs in defined folders that are named in convention with their lineage and segment (`output/Nextclade/H1N1/PB1/`).

- **Nextclade output files:**
    - `output/Nextclade/H1N1/HA/nextclade.csv` (Contains additional clade information)
    - `output/Nextclade/H1N1/HA/nextclade.aligned.fasta.fasta`
    - `output/Nextclade/H1N1/HA/nextclade_gene_HA1.translation.fasta`
    - `output/Nextclade/H1N1/HA/nextclade_gene_HA2.translation.fasta`
    - `output/Nextclade/H1N1/HA/nextclade_gene_SigPep.translation.fasta.fasta`

<br/>

 - **Nextalign output files:**
    - `output/Nextalign/H1N1/PA/nextalign.aligned.fasta`
    - `output/Nextalign/H1N1/PA/nextalign_gene_PA.translation.fasta`



2. Dataframe or Excel sheet with listing what amino acids are present for given positions per sequence in the final protein alignments. For HA and NA clades could be included to complement the output data of AA's per position.

## NOTE | Conda Configuration

### Conda set-up within arm-64 installation of conda.
Conda packages are installed within an osx-arm64 architecture (Intel) subdirectory of conda installation that runs osx-arm64 architecture (Apple M1/2).
In order for the `Snakemake`, `Nextclade` and `Nextalign` packages to work, they should run using a Conda environment that supports osx-64 packages.

This is where we tell Conda that we want our environment to use x86 architecture rather than the native ARM64 architecture. We can do that with following lines of code, which create and activate a new conda environment called `berlin`:

```
CONDA_SUBDIR=osx-64 conda create --name berlin python=3.10
conda activate berlin
conda config --env --set subdir osx-64
```

- The first line creates the environment. We simply set the CONDA_SUBDIR environment variable to indicate that conda should create the ennvironment with an osx-64 Python executable. 
- The second line activates the environment, and the third line ensures that conda installs osx-64 versions of packages into the environment.

> This new environment will now be able to use dependencies with osx-64 builds.

## Conda packages

#### Git:
```
conda install -c anaconda git
```

#### Jupyter:
```
conda install -c anaconda jupyter
```

#### Snakemake (osx-64):
```
conda install -c bioconda snakemake
```

#### Nextalign (osx-64):
```
conda install -c bioconda nextalign
```

#### Nextclade (osx-64):
```
conda install -c bioconda nextclade  
```

#### Graphviz (extension for Snakemake):
Helper tool to visualize the DAG (Directed Acyclic Graph) and save it as a .png file.

*Source:*
- Youtube | [An introduction to Snakemake tutorial for beginners (CC248)](https://www.youtube.com/watch?v=r9PWnEmz_tc)

```
conda install graphviz
```

To use the tool on the Snakemake file, run:

```
snakemake --dag targets | dot Tpng > dag.png
```

> NOTE: In the video the ```rule targets``` was made, listing all the outputs that need to be generated when running the basic snakemake command: 

```snakemake -c 1``` (runs using one thread)

# Code chunks

## Retrieve positions of interest for given lineages and seqments.
- Use pandas to read an excel file that lists positions per lineage and segment.

    Reminder: the file `input/positions_by_lineage_and_segment.xlsx` has the folowing columns:

lineage | segment | position
--------|---------|---------
H1N1pdm | HA      | 156
H1N1pdm | HA      | 158
H1N1pdm | HA      | 162
H1N1pdm | NA      | 274
H1N1pdm | PA      | 31
H3N2    | HA      | 145
H3N2    | HA      | 184
H3N2    | HA      | 185
etc...  | etc...  | etc...

<br>

- Import data from `input/positions_by_lineage_and_segments.xlsx` to get the desired positions to look for, within the AA sequence for a given lineage and segment.

In [1]:
# Import libraries:
import pandas as pd
from Bio import SeqIO

#########################################################################################
#                                                                                       #
# FUNCTION: Get headers and amino acid sequences from Nextaligns output files.          #
#                                                                                       #
#########################################################################################

def get_sequence_names_and_amino_acid_sequences(fasta_file):

    # Read the FASTA file and extract the nucleotide sequences.
    records = list(SeqIO.parse(fasta_file, "fasta"))

    # If there are no sequences in the fasta file, abort the function.
    if len(records) < 1:
        print("No sequences found in the FASTA file.")
        return

    # Get a list of sequence names from the headers.
    sequence_names = []
    for record in records:
        sequence_names.append(record.id)

    # Get a list of corresponding amino acid sequences
    amino_acid_sequences = []
    for record in records:
        amino_acid_sequences.append(record.seq)

    return sequence_names, amino_acid_sequences


#########################################################################################
#                                                                                       #
# FUNCTION: Get a list of positions of interest for lineage and segment of interest.    #
#                                                                                       #
#########################################################################################

def get_positions_from_excel(excel_file, lineage, segment):

    # Import pandas
    import pandas as pd

    # Use pandas to read columns from the Excel file
    position_by_lineage_and_segment = pd.read_excel(excel_file)

    # Make a 'selection' by filtering data to match positions for 'ha' segments belonging to the 'vic'lineage.
    selection = position_by_lineage_and_segment.query(
        f'lineage=="{lineage}" & segment=="{segment}"')

    # Get the selected values form the 'positions' columns as a list.
    positions = selection['position'].to_list()

    return positions


#########################################################################################
#                                                                                       #
# FUNCTION: Make an Excel file listing headers with amino acids at given positions.     #
#                                                                                       #
#########################################################################################

def generate_excel_output(output, sequence_names, amino_acid_sequences, positions):

    data = []
    for sequence_name, amino_acid_sequence in zip(sequence_names, amino_acid_sequences):
        row = [sequence_name]
        row.extend(amino_acid_sequence[pos - 1] for pos in positions)
        data.append(row)

    column_names = ["Sequence"] + [f"Pos {pos}" for pos in positions]

    df = pd.DataFrame(data, columns=column_names)

    df.to_excel(output, index=False)


#########################################################################################
#                                                                                       #
#  Run the functions above...                                                           #
#                                                                                       #
#########################################################################################



#
# -----------------------------    HEADERS & AA SEQUENCES   ----------------------------
#
# Set input fasta file (SNAKEFILE):
aa_fasta = '_nextclade/output/flu_vic_ha/nextclade_gene_HA1.translation.fasta'

# Get 'aa_names' and 'aa_sequences' from input fasta file:
aa_names, aa_sequences = get_sequence_names_and_amino_acid_sequences(
                            fasta_file=aa_fasta
                            )






#
# --------------------------    FILTER POSITIONS OF INTEREST   ------------------------
#
# Set 'positions' using the input Excel file and by defining filters (SNAKEFILE):
positions_table = 'input/positions_by_lineage_and_segment.xlsx'

filter_lineage = 'vic'

filter_segment = 'ha'

# Get the input positions from the input excel_file that are filtered by lineage and segment:
filtered_positions = get_positions_from_excel(
                        excel_file=positions_table,
                        lineage=filter_lineage,
                        segment=filter_segment
                        )






#
# -------------------------    CREATE OUTPUT EXCEL FILE   -----------------------------
#
# Set the name of the final output file (SNAKEFILE):
excel_output = 'output/aa_at_positions_for_vic_HA1.xlsx'

# Create an Excel file as final output:
if aa_names and aa_sequences:
    generate_excel_output(
        output=excel_output,
        sequence_names=aa_names,
        amino_acid_sequences=aa_sequences,
        positions=filtered_positions
    )
